### 練習問題2.19
1.2.2節の両替パターン数計算プログラムについ て考える。プログラムで使う通貨を変えられるようにして、例え ばイギリスポンドの両替パターン数を計算できるようになると便 利だろう。現状のプログラムでは、通貨についての知識は⼀部は first-denominationに、⼀部はcount-change (アメリカには5種 類のコインがあるということを知っている)に分散されている。両 替に使うコインのリストを与えられるようになればもっとよくな るだろう。 ⼿続きccを書き直して、⼆つ⽬の引数として取るものを、どのコ インを使うかを指定する整数ではなく、使うコインの値のリスト となるようにしたい。それから、それぞれの通貨の種類を定義す るリストを次のように持っておく。  

    (define us-coins (list 50 25 10 5 1))
    (define uk-coins (list 100 50 20 10 5 2 1 0.5))
    
そして、ccは次のように呼び出すことにしよう。  

    (cc 100 us-coins)
    292
    
このためには、プログラムccに対して何らかの変更を加える必要がある。プログラムの形は同じだが、⼆つ⽬の引数に対するアクセスの仕⽅が次のように変わる。

    (define (cc amount coin-values)
        (cond ((= amount 0) 1)
              ((or (< amount 0) (no-more? coin-values)) 0)
              (else
                  (+ (cc amount
                      (except-first-denomination coin-values))
                      (cc (- amount (first-denomination
                                      coin-values))
                          coin-values)))))
                      
リスト構造に対する基本演算によって、⼿続きfirst-denomination, except-first-denomination, no-more?を定義せよ。リストcoinvaluesの順序は、ccによって返される解答に影響を与えるだろう か、それとも与えないだろうか。それはどうしてだろうか。

In [1]:
(define (cc amount coin-values)
    (cond ((= amount 0) 1)
          ((or (< amount 0) (no-more? coin-values)) 0)
          (else
              (+ (cc amount (except-first-denomination coin-values))
                  (cc (- amount (first-denomination coin-values)) coin-values))
           )
    )
  )

; coin-values(リスト)が空どうかを判定する。
(define (no-more? coin-values)(null? coin-values))
; 最初のコイン種別を除いたコイン種別リストを返す。
(define (except-first-denomination coin-values)(cdr coin-values))
; 最初のコイン種別を返す。
(define (first-denomination coin-values)(car coin-values))

In [2]:
; 動作確認
(define us-coins (list 50 25 10 5 1))
(define uk-coins (list 100 50 20 10 5 2 1 0.5))
;(display us-coins)
(display (cc 11 us-coins))
(newline)
(display (cc 50 us-coins))
(newline)
(display (cc 100 us-coins))
(newline)

4
50
292


練習問題とは別に、  
コイン両替の組み合わせの動作確認をするために、  
以下の出力イメージで表示させたい。

    1:((50 . 0)(25 . 0)(10 . 1)(5 . 0)(1 . 1))
    2:((50 . 0)(25 . 0)(10 . 0)(5 . 2)(1 . 1))
    3:((50 . 0)(25 . 0)(10 . 0)(5 . 1)(1 . 6))
    4:((50 . 0)(25 . 0)(10 . 0)(5 . 0)(1 . 11))

In [3]:
; 以下、補助手続き
(define (make-pair type val)(cons type val))
(define (coin-type p)(car p))
(define (coin-val p)(cdr p))

; 与えられたリストに要素を追加したリストを返す。
(define (append-item l item)
  (if (null? l) (cons item '())
      (cons (car l) (append-item (cdr l) item)))  
  )

(display (append-item (list (make-pair 1 2) (make-pair 3 4) (make-pair 5 6)) (make-pair 7 8)))
(newline)
(display (append-item (list 1 2 3 4) 5))
(newline)

; 両替リストを更新する。
(define (update-list l p)
  (if (null? l) '()
      (let ((t (car l))
            (remain (cdr l))
            (new-type (coin-type p))
            (new-val (coin-val p))
            )
          (let ((type (coin-type t))
                (val (coin-val t))
                )
            (if (= new-type type)
                    (let ((new-p (make-pair new-type (+ new-val val))))
                      (cons new-p (update-list remain p))
                      )
                (cons t (update-list remain p))
                )
            )
        )
      )
  )

(display (update-list (list (make-pair 10 1) (make-pair 5 2) (make-pair 1 3)) (make-pair 5 7)))
(newline)

(define (make-list coin-values)
  (if (null? coin-values) '()
      (let ((p (make-pair (car coin-values) 0)))
          (cons p (make-list (cdr coin-values)))
        )
    )
  )
(define null-list (list))
(display null-list)
(newline)
(define us-coins (list 50 25 10 5 1))
(display us-coins)
(newline)
(display (make-list us-coins))
(newline)
;(display (length (make-list us-coins)))
;(newline)
(define (print-list l)
  (define (iter l-tmp cnt)
    (if (null? l-tmp) '()
        (let ((item (car l-tmp)))
          (begin
           (display cnt)
           (display ":")
           (display item)
           (newline)
           (iter (cdr l-tmp) (+ cnt 1))
           )
        )
      )
    )
    (iter l 1)
  )
(print-list (make-list us-coins))
;(newline)

((1 . 2) (3 . 4) (5 . 6) (7 . 8))
(1 2 3 4 5)
((10 . 1) (5 . 9) (1 . 3))
()
(50 25 10 5 1)
((50 . 0) (25 . 0) (10 . 0) (5 . 0) (1 . 0))
1:(50 . 0)
2:(25 . 0)
3:(10 . 0)
4:(5 . 0)
5:(1 . 0)


()

In [4]:
(define (cc2 amount coin-values)
  (let ((total-list (list))
        (coin-list (make-list coin-values)))
    (define (iter a coins l)
      (cond ((= a 0) (begin
                      (set! total-list (append-item total-list l))
                      total-list))
            ((or (< a 0) (no-more? coins)) total-list)
            (else
              (begin
                 (let ((first (first-denomination coins)))
                   (let ((new-l (update-list l (make-pair first 1))))
                     (iter (- a first) coins new-l)
                     )
                   )
                 (iter a (except-first-denomination coins) l)
               )
             )
          )
      )
    (iter amount coin-values coin-list)
  )
)

; 動作確認
(define us-coins (list 50 25 10 5 1))
;(display us-coins)
;(newline)
(define comb-list (cc2 50 us-coins))
(print-list comb-list)
(newline)
(display (length comb-list))
(newline)

1:((50 . 1) (25 . 0) (10 . 0) (5 . 0) (1 . 0))
2:((50 . 0) (25 . 2) (10 . 0) (5 . 0) (1 . 0))
3:((50 . 0) (25 . 1) (10 . 2) (5 . 1) (1 . 0))
4:((50 . 0) (25 . 1) (10 . 2) (5 . 0) (1 . 5))
5:((50 . 0) (25 . 1) (10 . 1) (5 . 3) (1 . 0))
6:((50 . 0) (25 . 1) (10 . 1) (5 . 2) (1 . 5))
7:((50 . 0) (25 . 1) (10 . 1) (5 . 1) (1 . 10))
8:((50 . 0) (25 . 1) (10 . 1) (5 . 0) (1 . 15))
9:((50 . 0) (25 . 1) (10 . 0) (5 . 5) (1 . 0))
10:((50 . 0) (25 . 1) (10 . 0) (5 . 4) (1 . 5))
11:((50 . 0) (25 . 1) (10 . 0) (5 . 3) (1 . 10))
12:((50 . 0) (25 . 1) (10 . 0) (5 . 2) (1 . 15))
13:((50 . 0) (25 . 1) (10 . 0) (5 . 1) (1 . 20))
14:((50 . 0) (25 . 1) (10 . 0) (5 . 0) (1 . 25))
15:((50 . 0) (25 . 0) (10 . 5) (5 . 0) (1 . 0))
16:((50 . 0) (25 . 0) (10 . 4) (5 . 2) (1 . 0))
17:((50 . 0) (25 . 0) (10 . 4) (5 . 1) (1 . 5))
18:((50 . 0) (25 . 0) (10 . 4) (5 . 0) (1 . 10))
19:((50 . 0) (25 . 0) (10 . 3) (5 . 4) (1 . 0))
20:((50 . 0) (25 . 0) (10 . 3) (5 . 3) (1 . 5))
21:((50 . 0) (25 . 0) (10 . 3) (5 . 2) (1 